[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/adams/blob/master/demos/nlp/word-2-vec.ipynb)

# Word Embeddings and Word-to-Vec (W2V)
This demo notebook revisits the lecture on word embeddings and Google's word-to-vec algorithm. W2V, like backpropagation, is a very popular algorithm that enjoys much coverage in various blogs, youtube channels, etc. In case you appreciate some additional material to read-up on W2V, here here are some useful resources including,  
- [the original W2V paper](https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf)
- the beautiful ["Illustrated Word2vec" by Jay Alammar](https://jalammar.github.io/illustrated-word2vec/)
- the[W2V Tensorflow tutorial](https://www.tensorflow.org/tutorials/text/word2vec)

Last but not least, our main textbook features excellent chapters on word embeddings, W2V, and related algorithms inlcuding GloVe and Fasttext. You can find those parts in [Section 14 of Dive into Deep Learning](http://d2l.ai/chapter_natural-language-processing-pretraining/index.html)

Let's get started with our ADAMS demo.

## Training word-to-vec embeddings
When it comes to embeddings, the most common use case is to **download pre-trained embeddings** and employ these for some downstream tasks (with or without fine-tuning). The Keras *embedding layer* supports that use case very well, as we will see in a future demo on sentiment analysis. Since this demo aims at deepening our understanding of W2V, we focus on a different use case and demonstrate the training of **customer word embeddings** using our IMDB data. 

You could argue that the IMDB forum exhibits a specific type of speech or jargon, and that this justifies training word embeddings for this specific corpus. In practice, using pre-trained embeddings will almost surely give better results than training embeddings from zero. However, without going into too much detail of the pros and cons of pre-training your own embeddings versus employing pre-trained embeddings, perhaps with some finetuning, the point of this section is simply to showcase how you could train from scratch if you want to. To that end, we will use a library called `Gensim`. 

`Gensim` is a popular library for text processing. Although maybe even more geared toward topic modeling, it offers, among others, implementations of several algorithms to learn word embeddings including *W2V*, *GloVe*, and *Fasttext*. We demonstrate training W2V embeddings using our cleaned IMDB movie review data set. Before moving on, make sure to have installed `Gensim`. 

**Credits and disclaimers**: many of the examples you are going to see in this section have been inspired by this very nice [Kaggle post](https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook).

In [98]:
# Create a global variable to idicate whether the notebook is run in Colab
import sys
import numpy as np
import pandas as pd

IN_COLAB = 'google.colab' in sys.modules

# Configure variables pointing to directories and stored files 
if IN_COLAB:
    # Mount Google-Drive
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_DIR = '/content/drive/My Drive/'  # adjust to Google drive folder with the data if applicable
else:
    DATA_DIR = './' # adjust to the directory where data is stored on your machine (if running the notebook locally)

sys.path.append(DATA_DIR)

CLEAN_REVIEW = DATA_DIR + 'imdb_clean_full_v2.pkl'   # List with tokenized reviews after standard NLP preparation
IMBD_EMBEDDINGS = DATA_DIR + 'w2v_imdb_full_d100_e500.model'

### Recap W2V
Let's quickly revisit the principles of W2V. Please consult the paper of [Mikolov et al. (2013)](https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf) for a detailed description.

W2V establishes a word's meaning by the words that frequently appear close-by (distributional semantics). More specifically, the context of a word consists of the words that appear next to it within a pre-defined window (let's say 5 words).

 - the quality of *air* in mainland China has been decreasing since..
 - doctors claim the *air* you breath defines the overall wellbeing...
 - the currents of hot *air* have been bursting from underground
 - the mountain *air* was crystal clean and filled with ..
 - in case of *air* supply shortages, the submarine will..

Taking the word *air* as our **target word**, the words around *air*, called context words, define the **meaning** of the word *air* in W2V.

![w2vprocess](w2v.jpg)
<br>
inspired by https://www.youtube.com/watch?v=BD8wPsr_DAI

### Loading the data
We load the data frame with the original and cleaned reviews. The original version does not matter for this session. We will delete them to save memory. 

In [17]:
%precision 

import pandas as pd 
df = pd.read_csv('train.csv')


df.isnull().sum()/len(df)*100

name                            0.025324
summary                         5.343318
space                          30.535055
description                     3.122061
experiences_offered             0.000000
neighborhood_overview          35.283265
transit                        35.827726
house_rules                    42.287099
picture_url                     0.000000
host_id                         0.000000
host_since                      0.200781
host_response_time             32.200998
host_response_rate             32.200998
host_is_superhost               0.200781
host_total_listings_count       0.200781
host_has_profile_pic            0.200781
host_identity_verified          0.200781
neighbourhood                   0.265900
neighbourhood_cleansed          0.000000
zipcode                         2.300847
latitude                        0.000000
longitude                       0.000000
property_type                   0.000000
room_type                       0.000000
accommodates    

In [22]:
text_cols = ['name', 'summary', 'space', 'description','neighborhood_overview', 'transit', 'house_rules']
text_df = df[[c for c in text_cols]]
text_df.loc[(df.description.isnull()),:].head(5)

,name,summary,space,description,neighborhood_overview,transit,house_rules
2,Comfi apartment 2 close to wimbledon tennis court,NaN,NaN,NaN,NaN,NaN,1- No party or event allowed. 2-No smoking ins...
8,Cosy room near Heathrow in Semi detached property,NaN,NaN,NaN,NaN,NaN,Guest should clean their dishes. Guest should ...
40,Big double room close to Stratford for 4 people,NaN,NaN,NaN,NaN,NaN,NaN
63,Cozy single room in North London,NaN,NaN,NaN,NaN,NaN,NaN
104,Double bedroom close to station,NaN,NaN,NaN,NaN,NaN,Guest sandals provided - to wear whilst inside...


In [162]:
## create corpus

import re

corpus_raw = text_df.description.dropna().tolist() + text_df.neighborhood_overview.dropna().tolist()
corpus =[]
for x in corpus_raw:
    temp = re.split(r'[.?!]',x.lower())
    temp = [x for x in temp if len(x.strip())>1]
    corpus.extend(temp[:-1])
    
corpus = list(set(corpus))

print(corpus[:5])

print(len(corpus))


[' great space for relaxing', 'always wanted to spend a few days like a real west londoner', ' and on the first floor a kitchen which you will all have full access to', ' traditional victorian house with modern facilities and everything you need for a comfortable stay', ' for foodies, just a stroll away, "l\'oculto" tapas and wine shop on brockley road is a good spot']
404816


In [145]:
import pickle

with open('corpus.pkl','wb') as f:
    pickle.dump(corpus, f)

### The Gensim W2V model
Training word embeddings using `Gensim` is very easy and just a matter of calling a function. Well, the reason it takes so little code is that we have already cleaned our data and have it available as an array of texts; that is a format that `Gensim`supports. However, note that, depending on your data, the code may take quite a while to run. Again, word embeddings trained on the full 50K data set for 500 epochs are available in our course folder.

Gensim is build for scalability. Would we use a large corpus, it were not be practical to first load all data from disk into your computer's main memory, to then process the data document by document using Gensim. Instead, it would be much more scalable to stream the data from disk. Long story short, we need a bit of infrastructure to input our review data set, which, for simplicity, we keep in a data frame, in a way that complies with what Gensim expects.  into. To that end, we build a little helper class that facilitates streaming reviews from our data frame. It would be easy to extend the helper class so as to facilitate streaming reviews from disk, or support both options. The [Gensim documentation](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#training-your-own-model) provides an example.

And here is the simple call to the function `Word2Vec` that trains our custom word embeddings.

Make sure to check out the docstring of the `Word2Vec` function to discover how word vectors are trained by default. Importantly, the argument `sg` let's you chose between *skip-gram* and *cbow*. Other concepts we discussed in the lecture include accelerating computations using *hierarchical softmax* and *negative sampling*. Gensim features these through its arguments `hs` and `negative`, respectively. Obviously, tons of other functionality is available, so make sure to study the [documentation](https://radimrehurek.com/gensim/models/word2vec.html?highlight=word2vec) if you plan to use the Gensim library for serious projects. Also, just to remind you, the [Kaggle post](https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook), which inspired this notebook, has a slightly more elaborate demo of how to set up training and, specifically, how you can break down the individual steps of W2V training into smaller pieces.

The trained word vectors are accessible through the field `wv` of the model class.

In [158]:
from gensim import utils
from gensim.models.phrases import Phrases,Phraser

sent = [utils.simple_preprocess(x) for x in corpus]
print(len(sent))

phrases = Phrases(sent, min_count=30, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]

for i, s in enumerate(sentences):
    print(s)
    if i>5:
        break

In [161]:
from gensim.models.callbacks import CallbackAny2Vec


class CleanReviews:
    """An iterator that yields sentences (lists of str)."""
    
    def __init__(self, reviews):
        self.reviews = reviews
        
    def __iter__(self):
        for line in self.reviews:
            yield utils.simple_preprocess(line)
            

            
class LossCallback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print('Loss after epoch {}: {}'.format(self.epoch, loss))
        self.epoch += 1

445366

In [160]:
# CAUTION: Running the code might take a while
from gensim.models.word2vec import Word2Vec    

emb_dim = 100  # embedding dimension, we use 10 for a quick demo of the code
reviews = CleanReviews(corpus)

# Train a Word2Vec model
model = Word2Vec(
    sentences=sentences,
    min_count=10,
    window=3, 
    sg=1,
    sample=1e-4,
    negative=10,
    epochs=20,
    vector_size=emb_dim,
    compute_loss=False,
    callbacks=[LossCallback()],
    batch_words=1000,
    workers=-1)    


['great', 'space', 'for', 'relaxing']
['always', 'wanted', 'to', 'spend', 'few', 'days', 'like', 'real', 'west', 'londoner']
['and', 'on', 'the', 'first_floor', 'kitchen', 'which', 'you', 'will', 'all', 'have', 'full', 'access', 'to']
['traditional', 'victorian', 'house', 'with', 'modern', 'facilities', 'and', 'everything', 'you', 'need', 'for', 'comfortable', 'stay']
['for', 'foodies', 'just', 'stroll', 'away', 'oculto', 'tapas', 'and', 'wine', 'shop', 'on', 'brockley', 'road', 'is', 'good', 'spot']
['my', 'flat', 'is', 'extremely', 'well', 'located', 'for', 'all', 'the', 'amenities', 'of', 'parsons_green']
['upon_arrival', 'receive', 'the', 'house', 'manual', 'with', 'the', 'latest', 'art', 'shows', 'galleries', 'gigs']
['the', 'neighbourhood', 'has', 'two', 'sides']
['we', 'have', 'high_ceilings', 'large', 'windows', 'double_glazed', 'windows', 'comfortable', 'beds', 'and', 'fully_equipped', 'kitchen']
['it', 'lovely', 'village', 'with', 'easy', 'rail', 'access', 'to', 'central', 'l

In [ ]:
print(model.wv.similarity('good', 'bad'))
print(model.wv.similarity('bed', 'great'))
print(len(model.wv.key_to_index))
print(model.wv.most_similar(positive=['room']))


In [ ]:
file="w2v_tmp.model"
save_as_bin = False
model.wv.save_word2vec_format(file, binary=save_as_bin)  # set binary to True to save disk space; false facilitates inspecting the embeddings in a text editor

In [ ]:
frozen_model = phrases.freeze()
frozen_model.save("./bigram_model.pkl")
model_reloaded = Phrases.load("./bigram_model.pkl")

In [ ]:
len(model.wv.key_to_index)  # how many word vectors have been trained
# model.wv.key_to_index

We continue with playing with word vectors shortly but let us first discuss input and output handling with Gensim.

### Input / output handling
Gensim supports saving and loading of trained embeddings in different versions. This makes a lot of sense since training can take a long time. For example, you could train for a couple of epochs, then store your results on disk, and then continue training. Here is how we can store our trained word vectors.

For Adams, you can obtain word vectors trained on the IMDB corpus for 500 epochs from our [GitHub repository](https://github.com/Humboldt-WI/adams/tree/master/demos/nlp). These vectors are far from comparable to real pre-trained W2V embeddings. On the other hand, their training took a couple of hours so the vectors should carry a bit more information compared to just running the above training code with a small embedding dimension of ten and training for only five epochs. Let's showcase how we can save and load word vectors.

In [58]:
# Load model from disk
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format(IMBD_EMBEDDINGS, binary=False)

Remember that you can also access the `KeyedVectors`, which we load with the previous statement, directly from a trained model object via the field `wv`. Thus, if you would like to run the following demos with the word vectors you trained yourself, simply run the following command. One would expect that the demos give nicer results with the pre-trained embeddings from your repo but you are welcoem to try this our yourself. 

In [65]:
# w2v = model.wv  # continue with the W2V embeddings trained above 